import os
os.environ['PATH']
stream = os.popen('owltools --help')
output = stream.read()
output

In [71]:
import pandas as pd
import os
from rdflib import Graph
from urllib.request import urlopen
import subprocess
import time
from IPython.display import Markdown as md

# use ontobio!!


In [72]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Mondo analysis

from ontobio.ontol_factory import OntologyFactory
mondo_file="/Users/matentzn/ws/mondo/src/ontology/mondo-qc.obo"
ofa = OntologyFactory()
ont = ofa.create(mondo_file)
ont.get_roots()[:3]

In [73]:
#sparql_dir_remote="https://raw.githubusercontent.com/monarch-initiative/mondo/no-subclass-between-genetic-disease/src/sparql/"
#mondo = "mondo-qc.owl"
#sparql_dir_local="../sparql/"
reports_dir_local="reports/"
warnings ="single-child no-subclass-between-genetic-disease related-exact-synonym excluded-subsumption-is-inferred".split(" ")
warning_reports = [warn+"-warning" for warn in warnings] 

mondo="http://purl.obolibrary.org/obo/mondo.owl"
g = Graph()
g.parse("../mondo-qc.owl", format="xml")

In [74]:
def sparql_path(sparql):
    return os.path.join(sparql_dir_local,sparql+".sparql")

def report_path(report):
    return os.path.join(reports_dir_local,report+".tsv")

def get_report(name):
    try:
        return pd.read_csv(report_path(name),sep="\t")
    except pd.errors.EmptyDataError:
        print("WARNING! ", name, " is empty and has been skipped.")
    return pd.DataFrame(columns=['EMPTY'])
        
def load_from_uri(uri):
    query = urlopen(sparql_iri).read().decode('utf-8')
    return query


def load_from_local_file(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    return data


## Robot report analysis

In [75]:
df = get_report("obo-report")
# Bug in ROBOT, wrong number of columns:
df.columns=['Rule Name', 'Subject', 'Property', 'Value','Trash']
df['Level'] = df.index
df.reset_index(level=0, inplace=True)
del df['index']
del df['Trash']

In [ ]:
### Overview of errors

This section aims 

In [76]:
df=df[df['Subject'].str.startswith('MONDO')]
df[['Level','Rule Name','Value']].groupby(['Level','Rule Name']).count()

### Overview of errors

Value
Level Rule Name                         
ERROR deprecated_class_reference      54
      multiple_equivalent_classes  37563
INFO  lowercase_definition           300
WARN  duplicate_exact_synonym       1655
      duplicate_scoped_synonym      1595
      equivalent_pair              37603
      missing_definition               0

In [77]:
for rule in df['Rule Name'].unique():
    display(md("#### %s" % rule))
    display(df[df['Rule Name']==rule].head())

# Revealed a more complex issue we need to sort first: https://github.com/monarch-initiative/mondo/issues/1979
# https://github.com/ontodev/robot/issues/733 # Ignore 0021108 0045024 until ticket resolved.

#### multiple_equivalent_classes

,Rule Name,Subject,Property,Value,Level
0,multiple_equivalent_classes,MONDO:0000001,owl:equivalentClass,NCIT:C2991,ERROR
1,multiple_equivalent_classes,MONDO:0000001,owl:equivalentClass,OGMS:0000031,ERROR
2,multiple_equivalent_classes,MONDO:0000001,owl:equivalentClass,SCTID:64572001,ERROR
3,multiple_equivalent_classes,MONDO:0000001,owl:equivalentClass,http://www.ebi.ac.uk/efo/EFO_0000408,ERROR
4,multiple_equivalent_classes,MONDO:0000001,owl:equivalentClass,http://www.orpha.net/ORDO/Orphanet_377788,ERROR


#### deprecated_class_reference

,Rule Name,Subject,Property,Value,Level
37816,deprecated_class_reference,MONDO:0000747,owl:equivalentClass,http://linkedlifedata.com/resource/umls/id/C00...,ERROR
37817,deprecated_class_reference,MONDO:0000747,owl:equivalentClass,COHD:4245842,ERROR
37818,deprecated_class_reference,MONDO:0000747,owl:equivalentClass,DOID:0060321,ERROR
37819,deprecated_class_reference,MONDO:0000747,owl:equivalentClass,ICD9:553.1,ERROR
37820,deprecated_class_reference,MONDO:0000747,owl:equivalentClass,MESH:D006554,ERROR


#### duplicate_exact_synonym

...

,Rule Name,Subject,Property,Value,Level
121277,missing_definition,MONDO:0000005,IAO:0000115,NaN,WARN
121278,missing_definition,MONDO:0000009,IAO:0000115,NaN,WARN
121279,missing_definition,MONDO:0000014,IAO:0000115,NaN,WARN
121280,missing_definition,MONDO:0000023,IAO:0000115,NaN,WARN
121281,missing_definition,MONDO:0000030,IAO:0000115,NaN,WARN


#### duplicate_scoped_synonym

,Rule Name,Subject,Property,Value,Level
128494,duplicate_scoped_synonym,MONDO:0000155,oboInOwl:hasExactSynonym,rare inborn error of sequestering of triglyceride,WARN
128495,duplicate_scoped_synonym,MONDO:0000155,oboInOwl:hasRelatedSynonym,rare inborn error of sequestering of triglyceride,WARN
128496,duplicate_scoped_synonym,MONDO:0000242,oboInOwl:hasExactSynonym,dermatophytosis of beard,WARN
128497,duplicate_scoped_synonym,MONDO:0000242,oboInOwl:hasRelatedSynonym,dermatophytosis of beard,WARN
128498,duplicate_scoped_synonym,MONDO:0000270,oboInOwl:hasExactSynonym,disorder of lower respiratory tract,WARN


#### lowercase_definition

,Rule Name,Subject,Property,Value,Level
130090,lowercase_definition,MONDO:0000712,IAO:0000115,frontotemporal dementia plus amyotrophic later...,INFO
130091,lowercase_definition,MONDO:0001554,IAO:0000115,secondary glaucoma caused by either excessive ...,INFO
130092,lowercase_definition,MONDO:0001709,IAO:0000115,sarcoidosis with a complication of hypercalcemia.,INFO
130093,lowercase_definition,MONDO:0001875,IAO:0000115,inflammation of the lateral epicondyle.,INFO
130094,lowercase_definition,MONDO:0002121,IAO:0000115,neuritis of a single nerve.,INFO


# Other checks analyses

In [80]:
for r in warning_reports:
    df_r = get_report(r)
    if (len(df_r)>0):
        display(md("#### %s" % r))
        display(df_r.head())

#### single-child-warning

,?cls,?total
0,<http://purl.obolibrary.org/obo/MONDO_0016094>,1
1,<http://purl.obolibrary.org/obo/MONDO_0021396>,1
2,<http://purl.obolibrary.org/obo/MONDO_0016093>,1
3,<http://purl.obolibrary.org/obo/MONDO_0008882>,1
4,<http://purl.obolibrary.org/obo/MONDO_0006224>,1


#### no-subclass-between-genetic-disease-warning

,?d1,?d1_label,?d2,?d2_label
0,<http://purl.obolibrary.org/obo/MONDO_0013529>,catecholaminergic polymorphic ventricular tach...,<http://purl.obolibrary.org/obo/MONDO_0008648>,"ventricular tachycardia, familial"
1,<http://purl.obolibrary.org/obo/MONDO_0013966>,catecholaminergic polymorphic ventricular tach...,<http://purl.obolibrary.org/obo/MONDO_0008648>,"ventricular tachycardia, familial"
2,<http://purl.obolibrary.org/obo/MONDO_0009597>,"metaphyseal chondrodysplasia, Spahr type",<http://purl.obolibrary.org/obo/MONDO_0009943>,Pyle disease
3,<http://purl.obolibrary.org/obo/MONDO_0012762>,catecholaminergic polymorphic ventricular tach...,<http://purl.obolibrary.org/obo/MONDO_0008648>,"ventricular tachycardia, familial"
4,<http://purl.obolibrary.org/obo/MONDO_0011211>,axial spondylometaphyseal dysplasia,<http://purl.obolibrary.org/obo/MONDO_0009943>,Pyle disease


#### related-exact-synonym-warning

,?cls,?label,?related
0,<http://purl.obolibrary.org/obo/MONDO_0014756>,"tremor, hereditary essential, 5",ETM5
1,<http://purl.obolibrary.org/obo/MONDO_0006999>,tooth disease,disorder of calcareous tooth
2,<http://purl.obolibrary.org/obo/MONDO_0009424>,Bartter disease type 2,hyperprostaglandin E syndrome 2
3,<http://purl.obolibrary.org/obo/MONDO_0017686>,inborn aminoacylase deficiency,rare inborn error of aminoacylase activity
4,<http://purl.obolibrary.org/obo/MONDO_0020680>,acute bronchiolitis,acute bronchiolitis


WARNING!  excluded-subsumption-is-inferred-warning  is empty and has been skipped.
